## Toy data set

Let's train a network to classify ice hockey match scores to match results. We'll have the teams' goal amounts as the input and one-hot encoding of HOME_WIN, DRAW and AWAY_WIN as output. For example, for a match that ends 6–4, we would have [6, 4] as the input and [1, 0, 0] as output, because it's a win for the home team.

In [ ]:
training_data = [
    ([5, 4], [1, 0, 0]),
    ([1, 4], [0, 0, 1]),
    ([2, 4], [0, 0, 1]),
    ([0, 0], [0, 1, 0]),
    ([2, 2], [0, 1, 0]),
    ([7, 1], [1, 0, 0]),
    ([1, 0], [1, 0, 0]),
    ([1, 2], [0, 0, 1])
]

test_data = [
    ([8, 2], [1, 0, 0]),
    ([7, 7], [0, 1, 0]),
    ([1, 3], [0, 0, 1]),
    ([3, 3], [0, 1, 0])
]

training_inputs, training_labels = zip(*training_data)
test_inputs, test_labels = zip(*test_data)

## Softmax classifier

Now let's use TensorFlow to build a softmax model for the task.

In [ ]:
import tensorflow as tf

# Clear the graph because we might run this
# cell in Jupyter many times, and we don't
# want multiple copies of the graph.
tf.python.framework.ops.reset_default_graph()

# Inputs
home_score = tf.placeholder(tf.float32, [])
away_score = tf.placeholder(tf.float32, [])

# Labels
home_win = tf.placeholder(tf.float32, [])
draw = tf.placeholder(tf.float32, [])
away_win = tf.placeholder(tf.float32, [])

# Weights
home_home = tf.Variable(0.0, name="home_home")
home_draw = tf.Variable(0.0, name="home_draw")
home_away = tf.Variable(0.0, name="home_away")
away_home = tf.Variable(0.0, name="away_home")
away_draw = tf.Variable(0.0, name="away_draw")
away_away = tf.Variable(0.0, name="away_away")

# Biases
home_bias = tf.Variable(0.0, name="home_bias")
draw_bias = tf.Variable(0.0, name="draw_bias")
away_bias = tf.Variable(0.0, name="away_bias")

# Outputs
with tf.name_scope("home_output"):
    home_output = home_score * home_home + away_score * away_home + home_bias
with tf.name_scope("draw_output"):
    draw_output = home_score * home_draw + away_score * away_draw + draw_bias
with tf.name_scope("away_output"):
    away_output = home_score * home_away + away_score * away_away + away_bias    

Now that we have built the graph, we can run it.

In [ ]:
with tf.Session() as session:
    session.run(tf.initialize_all_variables())
    print session.run(home_output, {home_score: 7, away_score: 3})

Let's see what the graph looks like in TensorBoard.

In [ ]:
with tf.Session() as session:
    writer = tf.train.SummaryWriter("../logs/softmax", graph_def=session.graph_def)
    writer.close()
    print "Done!"

Something like this:

![](images/softmax graph in tensorboard.png)

## Refactor the model to use 2D tensors

In [ ]:
import tensorflow as tf

# Clear the graph because we might run this
# cell in Jupyter many times, and we don't
# want multiple copies of the graph.
tf.python.framework.ops.reset_default_graph()

inputs = tf.placeholder(tf.float32, [None, 2], name="inputs")
expected_outputs = tf.placeholder(tf.float32, [None, 3], name="weights")
weights = tf.Variable(tf.zeros([2, 3], tf.float32), name="weights")
biases = tf.Variable(tf.zeros([3]), tf.float32, name="biases")

with tf.name_scope("linear_softmax"):
    outputs = tf.nn.softmax(tf.matmul(inputs, weights) + biases)

## Add gradient descent to the graph

In [ ]:
with tf.name_scope("cross_entropy"):
    cross_entropy = -tf.reduce_sum(expected_outputs*tf.log(outputs))

with tf.name_scope("train_step"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
    train_step = optimizer.minimize(cross_entropy)

In [ ]:
initialize = tf.initialize_all_variables()
labels = tf.argmax(outputs, 1)
expected_labels = tf.argmax(expected_outputs, 1)
correct_prediction = tf.equal(labels, expected_labels)

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## Train

In [ ]:
with tf.Session() as session:
    session.run(initialize)
    for step in xrange(1000):
        session.run(train_step, {
                inputs: training_inputs,
                expected_outputs: training_labels
            })

    print session.run(accuracy, {
                inputs: test_inputs,
                expected_outputs: test_labels
            })
    
    test_run_results = session.run(labels, {
            inputs: test_inputs
        })
    
    print zip(test_inputs, test_run_results)